# Named Entity Recognition Analysis

# Importing Libraries

In [17]:
# Import libraries
import pandas as pd
import numpy as np
import spacy
from spacy import displacy
import networkx as nx
import os
import matplotlib.pyplot as plt
import scipy
import re
from collections import Counter


In [4]:
# Download English module

!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ------------------ --------------------- 6.0/12.8 MB 33.5 MB/s eta 0:00:01
     ------------------------------ -------- 10.0/12.8 MB 34.4 MB/s eta 0:00:01
     --------------------------------------  12.6/12.8 MB 28.1 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 19.1 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [5]:
# Load spaCy's large English model
nlp = spacy.load("en_core_web_sm")

# Load the Twentieth-Century Text Data

In [6]:
# Load the text data
file_path = "20th_century_events_cleaned.txt"
with open(file_path, "r", encoding="utf-8") as file:
    text_data = file.read()

In [7]:
# Print a preview
print(text_data[:1000])  # First 1000 characters



Key events of the 20th century - Wikipedia

Personal tools
Donate Create account Log in
		Pages for logged out editors learn more
ContributionsTalk
Contents
move to sidebar
hide
(Top)
1
Historic events in the 20th century
Toggle Historic events in the 20th century subsection
1.1
World at the beginning of the century
1.1.1
"The war to end all wars": World War I (1914–1918)
1.1.2
Russian Revolution and communism
1.2
Between the wars
1.2.1
Economic depression
1.2.2
The rise of dictatorship
1.3
Global war: World War II (1939–1945)
1.3.1
The war in Europe
1.3.2
Blitzkrieg
1.3.3
Operation Barbarossa
1.3.4
Turning tides
1.3.5
Operation Overlord
1.3.6
Final days
1.3.7
The war in the Pacific
1.3.7.1
Background
1.3.8
Japanese Expansion
1.3.9
Allied offensive
1.3.10
Final days
1.3.11
The Holocaust
1.3.12
The Nuclear Age begins
1.4
The post-war world
1.4.1
The end of empires: decolonization
1.4.2
The Cold War (1947–1991)
1.4.3
War by proxy
1.4.4
The space race
1.4.5
The end of the Cold War
1.4.6


# Text Wrangling and Cleaning

In [8]:
# Remove special characters and multiple spaces
def clean_text(text):
    text = re.sub(r"[^a-zA-Z0-9.,;!?\'\"\s]", "", text)  # Remove unwanted characters
    text = re.sub(r"\s+", " ", text).strip()  # Remove extra spaces
    return text

In [32]:
    # Standardize country names
    country_variations = {
        "USA": "United States",
        "America": "United States",
        "UK": "United Kingdom",
        "Soviet Union": "Russia",
        "PRC": "China"
    }
    
    for variant, standard in country_variations.items():
        text = text.replace(variant, standard)
    
    return text

NameError: name 'text' is not defined

In [33]:
cleaned_text = clean_text(text_data)

In [36]:
# Save cleaned text to a new file
cleaned_file_path = "20th_century_events_cleaned_normalized.txt"
with open(cleaned_file_path, "w", encoding="utf-8") as file:
    file.write(cleaned_text)

In [37]:
# Print a preview
print(cleaned_text[:1000])

Key events of the 20th century Wikipedia Personal tools Donate Create account Log in Pages for logged out editors learn more ContributionsTalk Contents move to sidebar hide Top 1 Historic events in the 20th century Toggle Historic events in the 20th century subsection 1.1 World at the beginning of the century 1.1.1 "The war to end all wars" World War I 19141918 1.1.2 Russian Revolution and communism 1.2 Between the wars 1.2.1 Economic depression 1.2.2 The rise of dictatorship 1.3 Global war World War II 19391945 1.3.1 The war in Europe 1.3.2 Blitzkrieg 1.3.3 Operation Barbarossa 1.3.4 Turning tides 1.3.5 Operation Overlord 1.3.6 Final days 1.3.7 The war in the Pacific 1.3.7.1 Background 1.3.8 Japanese Expansion 1.3.9 Allied offensive 1.3.10 Final days 1.3.11 The Holocaust 1.3.12 The Nuclear Age begins 1.4 The postwar world 1.4.1 The end of empires decolonization 1.4.2 The Cold War 19471991 1.4.3 War by proxy 1.4.4 The space race 1.4.5 The end of the Cold War 1.4.6 Information and commu

#  Named Entity Recognition (NER) using spaCy

In [38]:
# Apply NLP processing
doc = nlp(cleaned_text)


In [39]:
# Extract named entities
entities = [(ent.text, ent.label_) for ent in doc.ents]

In [40]:
# Display first 20 entities
print(entities[:20])

[('the 20th century', 'DATE'), ('Wikipedia Personal', 'ORG'), ('Donate Create', 'PRODUCT'), ('ContributionsTalk Contents', 'PRODUCT'), ('1', 'CARDINAL'), ('the 20th century', 'DATE'), ('Toggle Historic', 'ORG'), ('the 20th century', 'DATE'), ('1.1', 'CARDINAL'), ('the beginning of the century', 'DATE'), ('World War I', 'EVENT'), ('19141918 1.1.2', 'DATE'), ('1.2', 'CARDINAL'), ('1.2.1', 'CARDINAL'), ('1.3', 'CARDINAL'), ('World War II', 'EVENT'), ('Europe', 'LOC'), ('Blitzkrieg 1.3.3 Operation Barbarossa 1.3.4', 'ORG'), ('Operation Overlord 1.3.6', 'EVENT'), ('Japanese Expansion', 'ORG')]


#  Extract and Filter Country Entities


In [41]:
# List of countries to filter (Use previous country lookup list if available)
countries = ["Germany", "Japan", "United States", "France", "Italy", "China", "Russia", "India", "United Kingdom", "Canada"]


In [42]:
# Filter only country entities
country_mentions = [ent.text for ent in doc.ents if ent.text in countries]


In [43]:
# Count occurrences
country_counts = Counter(country_mentions)

In [44]:
# Convert to DataFrame
df_countries = pd.DataFrame(country_counts.items(), columns=["Country", "Mentions"])

In [45]:
print(df_countries)  # Print the DataFrame in the console


         Country  Mentions
0         France        12
1         Russia         7
2        Germany        30
3          Italy        10
4          Japan        27
5          China         7
6          India         9
7         Canada         1
8  United States         1


# Create a Relationships DataFrame


In [46]:
# Extract sentences containing country mentions
sentences = [sent.text for sent in doc.sents if any(country in sent.text for country in countries)]

In [47]:
# Create relationships DataFrame
relationship_data = []
for sent in sentences:
    present_countries = [country for country in countries if country in sent]
    if len(present_countries) > 1:
        relationship_data.append({"Sentence": sent, "Countries": ", ".join(present_countries)})


In [48]:
df_relationships = pd.DataFrame(relationship_data)

In [49]:
# Display the DataFrame
print(df_relationships.head())  # Print first 5 rows

                                            Sentence               Countries
0  After a period of diplomatic and military esca...          France, Russia
1  The Bolsheviks negotiated the Treaty of BrestL...         Germany, Russia
2  In the treaty, Bolshevik Russia ceded the Balt...         Germany, Russia
3  Germany, 1933 Fascism first appeared in Italy ...          Germany, Italy
4  The Nazi Party in Germany was dedicated to the...  Germany, United States


#  Save and Export the DataFrame

In [50]:
# Save to CSV
df_relationships.to_csv("country_relationships.csv", index=False)


#  Push Notebook to GitHub